## Historic Analysis Demo

### Preliminaries

#### Setup IQS Connection

Execute the following piece of code to connect to the desired IncQuery Server instance.

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    address='http://localhost:8082',
    user='iqserver',
    password='iqs'
)

#### Select model

Select any revision in SST/Track 2 - Requirements V&V/master:

In [ ]:
selector = iqs.jupyter_tools.mms_commit_selector_widget()

Indexed commits in selected ref:

In [ ]:
def make_commit(value):
    result = selector.value()
    result.commit_id = value["commitId"]
    result.name = value["name"]
    return result

commits = [make_commit(value) for value in selector.commit_map.values()]
indexed_commits = [commit for commit in commits if commit.to_model_compartment().is_loaded_by_server(iqs)]

org_name = selector.org_map[selector.value().org_id].name
project_name = selector.project_map[selector.value().project_id].name
ref_name = selector.ref_map[selector.value().ref_id].name

indexed_commits

### Use reproducible model analysis

#### Find pre-registered model analysis

In [ ]:
analysis_config_name = "SAIC Validation Rules"
configs_with_given_name = [ config 
    for config in iqs.analysis.list_model_analysis_configurations().model_analysis_configuration_identifiers
    if config.configuration_name == analysis_config_name
]
if configs_with_given_name:
    analysis_config = configs_with_given_name[0]
    print ("Analysis configuration '{}' found. \nGo on.".format(analysis_config_name))
else:
    print ("Error: analysis configuration '{}' not found, must be pre-registered for this demo".format(analysis_config_name))

#### Execute analysis configuration on selected model

In [ ]:
from iqs_jupyter import schema

analysis_results = [ { 'model': commit, 'response': iqs.analysis.run_model_analysis(analysis_execution_request = schema.AnalysisExecutionRequest(
    configuration_id = analysis_config.configuration_id,
    compartment = commit.to_model_compartment()
))} for commit in indexed_commits]

commit_names = [ result["model"].name for result in analysis_results ]

analysis_results

### Visualization

#### Issue count by severity

In [ ]:
import plotly.offline as pyo
import plotly.graph_objects as go
import numpy as np

pyo.init_notebook_mode()

def get_kpi(result, severity):
  return next(kpi.value for kpi in result["response"].kpi_values if kpi.name == 'Total number of '+severity)

infos = [ get_kpi(result, "info") for result in analysis_results ]
errors = [ get_kpi(result, "errors") for result in analysis_results ]

fig = go.Figure()
fig.add_trace(go.Scatter(x=commit_names, y=infos, name='Infos'))
fig.add_trace(go.Scatter(x=commit_names, y=errors, name='Errors'))
fig.update_layout(title='Number of SAIC rule violations in<br>'+org_name+' > '+project_name+' > '+ref_name,
                   xaxis_title='Revision date',
                   yaxis_title='Problem count',
                   width = 500,
                   height = 2000
                 )
fig.show()

#### Issue count by rule

In [ ]:
import plotly.graph_objects as go
import numpy as np

def rules(result):
  return [ rule.configuration_rule.name for rule in result["response"].analysis_results ]

def count(result, rule_name):
  return next(len(rule.matches) for rule in result["response"].analysis_results if rule.configuration_rule.name == rule_name)

all_rules = list(set([ rule
    for result in analysis_results
    for rule in rules(result)
]))

fig = go.Figure()
for rule_name in all_rules:
  fig.add_trace(go.Bar(x=commit_names, y=[ count(result, rule_name) for result in analysis_results ], name=rule_name))
fig.update_layout(title='Number of SAIC rule violations in<br>'+org_name+' > '+project_name+' > '+ref_name,
                  xaxis_title='Revision date',
                  yaxis_title='Problem count',
                  height = 2000,
                  barmode='stack')
fig.show()